<a href="https://colab.research.google.com/github/Junyoung50403/Senior_Thesis/blob/main/TGA_Plotting_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies & Functions
Run this block before proceeding with anything else.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import os
import re
import textwrap

In [ ]:
# Matplotlib params
plt.rcParams["font.weight"] = "bold"  # Make all text bold
plt.rcParams["axes.labelweight"] = "bold"  # Bold x and y axis labels
plt.rcParams["axes.titleweight"] = "bold"  # Bold title
plt.rcParams['axes.labelsize'] = 14  # Increase x and y axis label size
plt.rcParams['xtick.labelsize'] = 12  # Increase x-axis tick labels
plt.rcParams['ytick.labelsize'] = 12  # Increase y-axis tick labels
plt.rcParams['axes.linewidth'] = 2  # Increase plot box border thickness
plt.rcParams["xtick.major.width"] = 2  # Make x-tick lines thicker
plt.rcParams["ytick.major.width"] = 2  # Make y-tick lines thicker
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
# Parses through one or more TGA data files, cleaning and saving all datasets #
def parse_runs():
  runs = {}
  names = {}
  masses = {}
  Hdens = {}
  water = {}
  i = 1
  uploaded_files = files.upload()

  keep = input("Use sheet names as labels? (y/n): ")

  for filename in uploaded_files.keys():
    sheets = pd.read_excel(filename, skiprows=4,sheet_name=None)
    for id in sheets.keys():
      df = sheets[id].copy()
      df = df.dropna(axis=1)
      df.drop(df.columns[2], axis=1, inplace=True)
      df.drop(df.columns[5], axis=1, inplace=True)
      df.columns = ["t", "wt", "dwt", "T", "Twtp", "Twt"]
      df.t = df.t / 60
      runs[len(runs) + 1] = df
      if (keep != "y"):
        names[len(names) + 1] = input("Enter a label for " + id + ": ")
      else:
        names[len(names) + 1] = id
      masses[len(masses) + 1] = df["Twt"][len(df["Twt"]) - 1]
      Hdens[len(Hdens) + 1] = float(input("Acid site density for " + id + ": "))
      i += 1

  return runs, names, masses, Hdens

In [ ]:
# Auxiliary function for shortening file names #
def short(filename):
    shortened_name = re.sub(r'^\d{4}\.\d{2}\.\d{2} ', '', filename)
    shortened_name = shortened_name.replace('.xlsx', '')
    return shortened_name

In [ ]:
# Produces a 2x2 subplot display for a given run #
def single_plot(df, x=7):
  fig, ax = plt.subplots(2, 2, figsize=(x, 10*(x/7)))
  ax[0, 0].plot(df.t, df.wt)
  ax[0, 0].set_title("Weight % vs. Time")
  ax[0, 0].set_xlabel("Time [h]")
  ax[0, 0].set_ylabel("Weight [%]")

  ax[0, 1].plot(df.t, df.dwt)
  ax[0, 1].set_title("Derivative Weight Loss vs. Time")
  ax[0, 1].set_xlabel("Time [h]")
  ax[0, 1].set_ylabel("Weight Loss [%/min]")

  ax[1, 0].plot(df["T"], df.Twtp)
  ax[1, 0].set_title("Weight % vs. Temperature")
  ax[1, 0].set_xlabel("Temperature [°C]")
  ax[1, 0].set_ylabel("Weight [%]")

  ax[1, 1].plot(df["T"], df.Twt)
  ax[1, 1].set_title("Weight vs. Temperature")
  ax[1, 1].set_xlabel("Temperature [°C]")
  ax[1, 1].set_ylabel("Weight [mg]")

  plt.tight_layout()

In [ ]:
# Produces an overlaid multiplot for given runs #
def multi_plot(runs, names, x=7):
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  fig, ax = plt.subplots(2, 2, figsize=(x, 10*(x/7)))
  ax[0, 0].set_title("Weight % vs. Time")
  ax[0, 0].set_xlabel("Time [h]")
  ax[0, 0].set_ylabel("Weight [%]")
  ax[0, 1].set_title("Derivative Weight Loss vs. Time")
  ax[0, 1].set_xlabel("Time [h]")
  ax[0, 1].set_ylabel("Weight Loss [%/min]")
  ax[1, 0].set_title("Weight % vs. Temperature")
  ax[1, 0].set_xlabel("Temperature [°C]")
  ax[1, 0].set_ylabel("Weight [%]")
  ax[1, 1].set_title("Weight vs. Temperature")
  ax[1, 1].set_xlabel("Temperature [°C]")
  ax[1, 1].set_ylabel("Weight [mg]")
  for i in ids:
    st = runs[i][runs[i]["T"] == 150].last_valid_index()
    ed = runs[i][runs[i]["T"] == 600].last_valid_index()
    st = 0

    edd = runs[i][runs[i]['t'] > runs[i].t[st] + 300].first_valid_index()

    ax[0, 0].plot(runs[i].t[st:ed] - runs[i].t[st], runs[i].wt[st:ed], label=names[i])
    ax[0, 1].plot(runs[i].t[st:ed] - runs[i].t[st], runs[i].dwt[st:ed], label=names[i])
    ax[1, 0].plot(runs[i]["T"][st:ed], runs[i].Twtp[st:ed], label=names[i])
    ax[1, 1].plot(runs[i]["T"][st:ed], runs[i].Twt[st:ed], label=names[i])
  plt.tight_layout()
  ax[0, 0].legend()
  ax[0, 1].legend()
  ax[1, 0].legend()
  ax[1, 1].legend()


In [ ]:
# Derivative Weight Loss Curve with Support for Averages
def dwts(runs, names, x=7):
    ids = input("Enter numbers separated by a space: ").split()
    window = int(input("Input end time for window: "))
    fig, ax = plt.subplots(figsize=(x, 10 * (x / 7)))
    ax.set_title("Derivative Weight Loss vs. Time")
    ax.set_xlabel("Time [min]")

    for i in ids:
        # Find the runs to average
        toAvg = i.split(',')
        toAvg = [int(a) for a in toAvg]

        count = 0
        total = None

        for tA in toAvg:
            count += 1
            if total is None:
                total = runs[tA].dwt
            else:
                total += runs[tA].dwt

        avg = total / count  # Average the dwt values across selected runs

        # Find the start and end index for plotting
        st = runs[toAvg[0]][runs[toAvg[0]]["T"] == 150].last_valid_index()
        ed = runs[toAvg[0]][runs[toAvg[0]]["T"] == 600].last_valid_index()

        dwted = runs[toAvg[0]][runs[toAvg[0]]['t'] > runs[toAvg[0]]['t'][st] + window].first_valid_index()

        # Find the index of the minimum value in the selected range
        min_idx = avg.iloc[st:dwted].idxmin()  # Get index of min value in avg

        # Get corresponding time for plotting
        min_time = runs[toAvg[0]].t[min_idx] - runs[toAvg[0]].t[st]
        min_value = avg[min_idx]

        # Plot the desired range using avg (average dwt) and time range
        ax.plot(runs[toAvg[0]].t.iloc[min_idx:dwted] - runs[toAvg[0]].t[st], avg.iloc[min_idx:dwted], label=names[toAvg[0]], alpha=1)

        # Plot the minimum point
        ax.scatter(min_time, min_value, zorder=3, marker='s')

    ax.invert_yaxis()
    plt.tight_layout()
    ax.legend()
    plt.show()

In [ ]:
def copy_until_first_occurrence(df):
    # Find the first occurrence of the value in the specified column
    first_occurrence_idx = df[df['T'] == 600].first_valid_index()
    return df.iloc[:first_occurrence_idx+1].copy()

In [ ]:
# Return notable indices (start, end, start_coke, end_coke) #
def find_indices_fixed(df, T):
    # Find the index where T == 250 (start of the reaction, if fixed 250 C rxn)
    start = df[df['T'] == T].first_valid_index()

    dfc = copy_until_first_occurrence(df)

    end = dfc[(dfc['dwt'] > -0.01) & (dfc['dwt'] < 0) & (dfc['T'] == T)].first_valid_index()
    if end == None:
        end = df[df['T'] == T].last_valid_index()


    coke = df[df['T'] == 600]

    stc = coke.first_valid_index()
    ecd = coke.last_valid_index()

    return start, end, stc, ecd

In [ ]:
# Return notable indices (start, end, start_coke, end_coke) #
def find_indices_ramp(df):
    # Find the index where T == 150 (start of the reaction, if linear ramp rxn)
    start = df[df['T'] == 150].last_valid_index()

    dfc = copy_until_first_occurrence(df)

    end = dfc[(dfc['dwt'] > -0.01) & (dfc['dwt'] < 0) & (dfc['T'] < 600)].last_valid_index()
    if end == None:
        end = df[df['T'] == 250].last_valid_index()


    coke = df[df['T'] == 600]

    stc = coke.first_valid_index()
    ecd = coke.last_valid_index()

    return start, end, stc, ecd

In [ ]:
def find_conversions(isFixed=True):
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []
  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0

    if (isFixed):
      st, ed, stc, edc = find_indices_fixed(runs[i], T)
    else:
      st, ed, stc, edc = find_indices_ramp(runs[i])

    name.append(names[i])
    conv.append((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / runs[i]['Twt'][st])
  # Create the bar chart
  fig, ax = plt.subplots(figsize=(7.2,4.8))
  bars = ax.bar(name, conv, color='skyblue', edgecolor='black', hatch='//', linewidth=2)  # Add edgecolor for outline

  ax = plt.gca()
  ax.set_xticks(range(len(name)))
  ax.set_xticklabels([textwrap.fill(n, width=10) for n in name], ha="center")

  # Set labels and title
  ax.set_xlabel('Catalyst')
  ax.set_ylabel('Solid Conversion [%]')
  ax.set_title('Solid Conversions')
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

In [ ]:
def find_coke(isFixed=True):
  feed = input("Reactant Molecule: ")
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []
  ratios = []
  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0

    if (isFixed):
      st, ed, stc, edc = find_indices_fixed(runs[i], T)
    else:
      st, ed, stc, edc = find_indices_ramp(runs[i])

    PE = runs[i]['Twt'][st] - runs[i]['Twt'][edc]
    ratios.append(PE / masses[i])

    name.append(names[i])
    conv.append((runs[i]['Twt'][stc] - runs[i]['Twt'][edc]) / masses[i])
  # Create the bar chart
  fig, ax = plt.subplots(figsize=(7.2,4.8))
  bars = ax.bar(name, conv, color='skyblue', edgecolor='black', hatch='//', linewidth=2)  # Add edgecolor for outline

  ax = plt.gca()
  ax.set_xticks(range(len(name)))
  ax.set_xticklabels([textwrap.fill(n, width=10) for n in name], ha="center")

  # Secondary Y-axis
  ax2 = ax.twinx()
  ax2.scatter(name, ratios, color='lightgray', marker='s', zorder=3, edgecolor='black', s=100, linewidth=1.2)
  ax2.set_ylabel(f"{feed}:Catalyst Ratio", color='gray')

  ax2.tick_params(axis='y', colors="gray")  # Change tick color
  ax2.spines["right"].set_color("gray")  # Change right spine color


  # Set labels and title
  ax.set_xlabel('Catalyst')
  ax.set_ylabel('Mass of coke (Mass of catalyst)^-1')
  ax.set_title('Coke Masses')
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

In [ ]:
# Normalized solid conversions
def norm_conversions(isFixed=True):
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []
  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0

    if (isFixed):
      st, ed, stc, edc = find_indices_fixed(runs[i], T)
    else:
      st, ed, stc, edc = find_indices_ramp(runs[i])

    name.append(names[i])
    conv.append(((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / 1000) / (masses[i] * (Hdens[i] / (1000 * 1000)) * ((runs[i]['t'][ed] - runs[i]['t'][st])/60)))
  # Create the bar chart
  fig, ax = plt.subplots(figsize=(7.2,4.8))
  bars = ax.bar(name, conv, color='skyblue', edgecolor='black', hatch='//', linewidth=2)  # Add edgecolor for outline

  ax = plt.gca()
  ax.set_xticks(range(len(name)))
  ax.set_xticklabels([textwrap.fill(n, width=10) for n in name], ha="center")

  # Set labels and title
  ax.set_xlabel('Catalyst')
  ax.set_ylabel('Solid Conversion Rate \n (g consumed) (mol BAS * h)^-1')
  ax.set_title('Conversion Rates')
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

In [ ]:
# Find partial conversion indices
# Return notable indices (start, end, start_coke, end_coke) #
def find_partial_indices_fixed(df, x, T=250):
    # Find the index where T == 250 (start of the reaction, if fixed 250 C rxn)
    start = df[df['T'] == T].first_valid_index()

    dfc = copy_until_first_occurrence(df)

    startWeight = df['Twt'][start]
    endWeight = startWeight * (1-x)

    end = dfc[(dfc['Twt'] <= endWeight) & (dfc['T'] == T)].first_valid_index()
    if end == None:
        end = df[df['T'] == T].last_valid_index()


    coke = df[df['T'] == 600]

    stc = coke.first_valid_index()
    ecd = coke.last_valid_index()

    return start, end, stc, ecd

In [ ]:
# Normalized PARTIAL solid conversions
def norm_partial_conversions():
  x = float(input("Input desired partial conversion fraction: "))
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []
  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0
    st, ed, stc, edc = find_partial_indices_fixed(runs[i], x, T)
    name.append(names[i])
    conv.append(((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / 1000) / (masses[i] * (Hdens[i] / (1000 * 1000)) * ((runs[i]['t'][ed] - runs[i]['t'][st])/60)))
  # Create the bar chart
  fig, ax = plt.subplots(figsize=(7.2,4.8))
  bars = ax.bar(name, conv, color='skyblue', edgecolor='black', hatch='//', linewidth=2)  # Add edgecolor for outline

  ax = plt.gca()
  ax.set_xticks(range(len(name)))
  ax.set_xticklabels([textwrap.fill(n, width=10) for n in name], ha="center")

  # Set labels and title
  ax.set_xlabel('Catalyst')
  ax.set_ylabel('Solid Conversion Rate \n (g consumed) (mol BAS * h)^-1')
  ax.set_title('Conversion Rates')
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

In [ ]:
# Conversion Rates vs. Conversion %
def conv_scatter(isFixed=True):
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []

  fig, ax = plt.subplots(figsize=(7.2,4.8))

  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0

    if (isFixed):
      st, ed, stc, edc = find_indices_fixed(runs[i], T)
    else:
      st, ed, stc, edc = find_indices_ramp(runs[i])

    norm_conv = (((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / 1000) / (masses[i] * (Hdens[i] / (1000 * 1000)) * (runs[i]['t'][ed] - runs[i]['t'][st])))

    conv = ((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / runs[i]['Twt'][st])

    ax.scatter(conv, norm_conv, label=names[i], linestyle='', marker='D')


  # Set labels and title
  ax.set_xlabel('Total Conversion (%)')
  ax.set_ylabel('Solid Conversion Rate \n (g consumed) (mol BAS * h)^-1')
  ax.set_title('Conversion Rates')
  ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

In [ ]:
# Normalized solid conversions
def water(isFixed=True):
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []
  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0

    if (isFixed):
      st, ed, stc, edc = find_indices_fixed(runs[i], T)
    else:
      st, ed, stc, edc = find_indices_ramp(runs[i], T)

    name.append(names[i])
    conv.append(((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / 1000) / (masses[i] * (Hdens[i] / (1000 * 1000)) * (runs[i]['t'][ed] - runs[i]['t'][st])))
  # Create the bar chart
  fig, ax = plt.subplots(figsize=(7.2,4.8))
  bars = ax.bar(name, conv, color='skyblue', edgecolor='black', hatch='//')  # Add edgecolor for outline

  ax = plt.gca()
  ax.set_xticks(range(len(name)))
  ax.set_xticklabels([textwrap.fill(n, width=10) for n in name], ha="center")

  # Set labels and title
  ax.set_xlabel('Catalyst')
  ax.set_ylabel('Solid Conversion Rate \n (g consumed) (mol BAS * h)^-1')
  ax.set_title('Conversion Rates')
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

In [ ]:
# Mass-normalized conversions
def mass_norm_conversions(isFixed=True):
  T = float(input("Input desired temperature: "))
  ids = input("Enter numbers separated by a space: ").split()
  ids = [int(id) for id in ids]
  name = []
  conv = []
  for i in ids:
    st, ed, stc, edc = 0, 0, 0, 0

    if (isFixed):
      st, ed, stc, edc = find_indices_fixed(runs[i], T)
    else:
      st, ed, stc, edc = find_indices_ramp(runs[i])

    name.append(names[i])
    conv.append(((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / 1000) / ((masses[i] / 1000) * ((runs[i]['t'][ed] - runs[i]['t'][st])/60)))
    print(((runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / 1000) / ((masses[i] / 1000) * ((runs[i]['t'][ed] - runs[i]['t'][st])/60)))
  # Create the bar chart
  fig, ax = plt.subplots(figsize=(7.2,4.8))
  bars = ax.bar(name, conv, color='skyblue', edgecolor='black', hatch='//', linewidth=2)  # Add edgecolor for outline

  ax = plt.gca()
  ax.set_xticks(range(len(name)))
  ax.set_xticklabels([textwrap.fill(n, width=10) for n in name], ha="center")

  # Set labels and title
  ax.set_xlabel('Catalyst')
  ax.set_ylabel('Solid Conversion Rate \n (g consumed) (g catalyst * h)^-1')
  ax.set_title('Conversion Rates')
  #ax.set_ylim(0.7,0.9)

  # Display the plot
  plt.show()

# Uploading TGA Data
The first block MUST be run, in order for files to be uploaded and processed. Afterwards, the second block is an optional block that lists ID numbers and associated TGA runs.

In [ ]:
# Run this block to upload one or more TGA data files #
runs, names, masses, Hdens = parse_runs()

In [ ]:
# Run this block to generate a directory of uploaded files #
count = 1
print("\033[4m#\033[0m","   \033[4mSample Name\033[0m")
for name in names:
  print(count,"::",names[name])
  count += 1

# Single Plots
Inbuilt support for single plots. Run the block to interactively produce a plot for a given TGA run. Do not modify the code (except to change the scaling constant); simply run the block to begin the process.

In [ ]:
# This function produces a single plot for the ID number entered when prompted #
# Change the constant argument to rescale the plot
single_plot(runs[int(input("Enter run number: "))], 7)

# Multi Plots
Inbuilt support for multi-plots. Run the block to interactively produce a set of graphs with overlaid plots from the chosen runs. Do not modify the code (except to change the scaling constant); simply run the block to begin the process.

In [ ]:
# This function overlays all plots of runs entered when prompted #
# Change the constant argument to rescale the plot
multi_plot(runs, names, 9)

# Higher-Order Data (Conversion, Coke, etc.)

In [ ]:
find_conversions()

In [ ]:
find_coke()

In [ ]:
norm_conversions()

# Debugging Tools

In [ ]:
for i in range(1, len(names) + 1):
  st, ed, stc, edc = find_indices_fixed(runs[i])
  print("Solid Conversion: ", (runs[i]['Twt'][st] - runs[i]['Twt'][ed]) / runs[i]['Twt'][st])

for i in range(1, len(names) + 1):
  st, ed, stc, edc = find_indices_fixed(runs[i])
  print("Coke Mass: ", (runs[i]['Twt'][stc] - runs[i]['Twt'][edc]))

for i in range(1, len(names) + 1):
  print(find_indices_fixed(runs[i]))
  print(Hdens[i])
  print(masses[i])

In [ ]:
norm_partial_conversions()

In [ ]:

dwts(runs, names, 5)

In [ ]:
conv_scatter(True)

In [ ]:
mass_norm_conversions()